# Secret Sharing

_Secret sharing_ protocols are ways to "split" a secret value (call it $s$) into pieces and distribute those pieces among a large set of people. Then, if some subset (or maybe the entire set) works together, they can recover $s$; otherwise, $s$ remains completely unknown.

First, we'll import some basic Python functions we'll use throughout our code:

In [ ]:
import numpy as np
import random, sympy, math
import matplotlib.pyplot as plt

## XOR Secret Sharing

XOR secret sharing is a simple 2-out-of-2 secret sharing scheme, meaning we split the secret $s$ into 2 _shares_ and both shares are required to recover the secret. The "XOR" part of the name refers to the eXclusive OR (XOR) operation, denoted by $\oplus$. Remember from our packet that this is a _binary_ operation described by the following table:

|||
|-------------|---|
|0 $\oplus$ 0 | 0 |
|0 $\oplus$ 1 | 1 |
|1 $\oplus$ 0 | 1 |
|1 $\oplus$ 1 | 0 |

In [ ]:
def int_to_bin(i):
    return bin(i)[2:]

def bin_to_int(b):
    return int('0b{}'.format(b), 2)

In [ ]:
print(int_to_bin(8))
print(bin_to_int('111'))

In [ ]:
print("bar")

Python has a built-in XOR operator: `^`. To compute $a \oplus b$ in Python, we can just write `a ^ b`:

In [ ]:
print(0^0)
print(0^1)
print(1^0)
print(1^1)

Even better, it works on integers, because Python automatically converts them into binary, computes the XOR, and converts the binary output back into an integer!

In [ ]:
17^42

### Share and Reconstruct
Okay, now let's use this to make a secret-sharing scheme. Here's what our sharing and reconstruction functions look like in Python:

In [ ]:
def xor_share(s):
    security_level = 2**8
    s1 = random.randint(0, security_level)
    s2 = s ^ s1
    return (s1, s2)

def xor_recon(s1, s2):
    return s1^s2

In [ ]:
s1, s2 = xor_share(42)
s1, s2

In [ ]:
xor_recon(s1, s2)

## Shamir Secret Sharing

Shamir secret sharing is a $(t+1)$-out-of-$n$ secret sharing protocol, for some numbers $t$ and $n$. This means that we split the secret $s$ into $n$ values and distribute them to $n$ people. Then, at least $t+1$ of those people must work together to recover $s$.

First, let's define some helper functions for dealing with polynomials.

**TODO: move these into another file and import them (the students don't need to see or mess with this code).**

In [ ]:
def term_to_string(coeff, deg):
    if coeff == 0:
        return ""

    temp = "{}".format(coeff)
    app = ""
    # constant term
    if deg == 0:
        return temp
    # x term
    elif deg == 1:
        app = "x"
    # others
    else:
        app = "x^{}".format(deg)
    return app if coeff == 1 else temp+app

def print_poly(coeffs):
    poly_str = ""
    
    # coefficients from highest to lowest degree
    deg = len(coeffs)-1
    for i in range(len(coeffs)):
        if(coeffs[i]!=0):
            poly_str += "{} + ".format(term_to_string(coeffs[i], deg-i))
    
    # remove extra + at end
    print(poly_str[:-3])
    
def eval_poly(coeffs, x):
    # coefficients from highest to lowest degree
    deg = len(coeffs)-1
    ans = 0
    for i in range(len(coeffs)):
        ans += coeffs[i]*x**(deg-i)
        
    # in real SSS, this is over a finite field (mod p)
    # return ans%p
    
    # but this is not representable in 2D so we will use int arith
    # (note this is not secure)
    return ans

### Share
Here is our sharing function:
- define a polynomial $f(X) = f_t X^t + \ldots + f_1 X + s$, where $f_t, \ldots, f_1$ are randomly chosen coefficients
- give the _share_ $f(i)$ to $i$th person (where $i$ is an index between 1 and $n$)

**TODO: describe extra details like the requirement that $t < n < p$ and that we're working over a field -- probably without actually using the term field.**

This is what it looks like in Python:

In [ ]:
def share(s, n, t, p):
    # check p
    if not sympy.isprime(p):
        print("p={} is not prime!".format(p))
        return
    if p <= n:
        print("p={} must be greater than n={}".format(p, n))
        return
        
    # check t
    if t >= n:
        print("t={} must be less than n={}".format(t, n))
        return

    # check s is in field
    if p <= s:
        print("s={} must be less than p={}".format(s, p))
        return
        
    coeffs = []
    for i in range(t):
        # sample coefficients from F_p = {0, ..., p-1}
        coeffs.append(random.randint(0,p-1))
    # secret is the y-intercept
    coeffs.append(s)
    
    shares = []
    for i in range(1,n+1):
        shares.append((i, eval_poly(coeffs, i)))
        
    # plot the polynomial
    print("The random degree t={} polynomial is".format(t))
    print_poly(coeffs)
    x = np.linspace(0, n, n+1)
    y = [eval_poly(coeffs, i) for i in x]
    plt.plot(x, y)
    
    # plot the shares
    print()
    print("The shares are points on that polynomial:")
    print(shares)
    x1 = [shares[i][0] for i in range(len(shares))]
    y1 = [shares[i][1] for i in range(len(shares))]
    plt.scatter(x1, y1)
    
    # plot the secret
    plt.scatter(0, s)
    
    print()
    print("Here is a visual representation (secret in orange).")
        
    return shares

For example, say our secret is the number 42. We'd like to share it among 10 parties (n=10), and we'll allow any 4 of those to recover the secret (t=3, t+1 can recover). Now let's pick a prime number p that's bigger than both the secret and the number of parties (so, p>42). The next largest prime is 43, so let's try that!

How would you call the `share` function with these parameters?

**Answer**
```
shares = share(42, 10, 3, 43)
```

In [ ]:
n=10
t=3
p=43
shares = share(42, n, t, p)

Now we can distribute these points among our 10 parties!

### Reconstruct
So now that we have shares, how can $t+1$ parties get back the secret? We can reconstruct $s$ by having at least $t+1$ parties pool their points $(i, f(i))$ and reconstruct the polynomial $f$. One way to do this (introduced in the packet) is called *Lagrange interpolation*:

$$ f(X) = \sum_i^{t+1} \ell_i \cdot f(i), \text{ where } \ell_i(X) = \frac{\Pi_{j \neq i} (X-x_j)}{\Pi_{j \neq i} (x_i-x_j)} $$ 

Once we've recovered the polynomial $f$, we can just evaluate $f(0) = s$. Here's how to do that in Python:

In [ ]:
def recon(shares, n, t):
    if len(shares) < t+1:
        print("Not enough shares to reconstruct! ({} < t+1={})".format(len(shares), t+1))
        return
    
    # i Lagrange basis polynomials evaluated at 0
    ell = [1]*len(shares)
    for i in range(len(shares)):
        #ell[i] = 1
        for j in range(len(shares)):
            if i!=j:
                ell[i] *= float(0-shares[j][0])/(shares[i][0]-shares[j][0])
    
    # interpolate
    # f(X) = sum_1^{t+1} ell_i(X) * y_i
    # s = f(0)
    s = 0
    for i in range(len(shares)):
        s += ell[i]*shares[i][1]

    print("The reconstructed secret is:")
    return int(s)

Say, for example, that 4 of our 10 parties above (Alice, Bob, Charlie, and Diana) want to recover the secret. We set $t=3$, so they should be able to do this (remember, a minimum of $t+1$ parties is needed). Together, they hold 4 points on the degree-3 polynomial, which uniquely defines it! They can pool this information to recover the polynomial $f$ and evaluate it at $x=0$ using `recon`.

Assuming Alice, Bob, Charlie, and Diane have the points for $x=1,2,3,4$, respectively, can you use the shares we created earlier to recover the secret?

**Answer**
```
recon(shares[:4], n=10, t=3)
```

In [ ]:
recon(shares, n, t)